In [183]:
import numpy as np
import cv2
np.random.seed(2)   # pour bloquer le random

# Définition des paramètres intrasèques de la caméra
  # focale
fx = 60
fy = 40
  # centre
cx = 0
cy = 0

A = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]) # matrice intrasèque de la caméra (3*3)

# Génération de 3 angles aléatoire pour définir la rotation
rax,pitch,yaw = np.random.uniform(0,np.pi,size=3)

# Génération de la matrice de rotation

Rx = np.array([[1,0,0],[0, np.cos(rax), -np.sin(rax)],[0, np.sin(rax), np.cos(rax)]])
Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],[0,1,0],[-np.sin(pitch),0, np.cos(pitch)]])
Rz = np.array([[np.cos(yaw), -np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0,0,1]])

R = Rz @ Ry @ Rx  # matrice de rotation (3*3)

# Génération de la translation de la caméra
T = np.random.uniform(-500,500,size=3)     # T = [tx,ty,tz]  (1*3)
T = T.reshape((3,1))                       # (3*1)

print(T)

[[ -64.67760738]
 [ -79.63219791]
 [-169.665179  ]]


In [184]:
# Projection d'un point 3D vers 2D
def projection3D2D(point3D,T,R,A) :
  # point 3D = [ Xw, Yw, Zw ]'   (1*3)
  # T : matrice de translation de la caméra : (3*1)
  # R : matrice de rotation : (3*3)
  # A : matrice intrasèque de la caméra : (3*3)

  PI = np.concatenate((np.eye(3),np.zeros((3,1))),axis=1)  # (3*4)

  Rt = np.concatenate((R,T),axis=1)               # (3*4)
  Rt = np.concatenate((Rt,np.array([[0,0,0,1]])),axis=0)   # (4*4)

  point3D_bis = np.concatenate((np.reshape(point3D,(3,1)),np.array([[1]])),axis=0)   #(4*1)

  point2D = A @ PI @ Rt @ point3D_bis   # point 2D = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # point 2D = [u, v, 1] (3*1)
  return point2D[:2]

pt3D = [[3],[1],[4]]
print("point 3D :\n ", pt3D)
pt2D = projection3D2D(pt3D,T,R,A)
print("point 2D :\n", pt2D)

point 3D :
  [[3], [1], [4]]
point 2D :
 [[21.94297343]
 [18.07044997]]


In [185]:
# Création des couples de point 2D,3D
def point3Daleatoire() :
  return np.array([[np.random.uniform(-500,500),np.random.uniform(-500,500),np.random.uniform(-500,500)]])

# points 3D
'''P1 = point3Daleatoire()     # (1*3) -> pour P3P
P2 = point3Daleatoire()
P3 = point3Daleatoire()
P4 = point3Daleatoire()'''
P1 = np.array([[-20,20,1]],dtype=np.float32)
P2 = np.array([[20,-20,1]],dtype=np.float32)
P3 = np.array([[-20,-20,1]],dtype=np.float32)
P4 = np.array([[20,20,1]],dtype=np.float32)

print(np.shape(P1))
print("P1\n",P1)
points3D = np.concatenate((P1,P2,P3),axis=0);     # (3*3)
print("pt3D\n",points3D)

# point 2D
p1 = projection3D2D(P1,T,R,A)   # (2*1)
p1 = np.reshape(p1,(1,2))       # (1,2)
p2 = projection3D2D(P2,T,R,A)
p2 = np.reshape(p2,(1,2))
p3 = projection3D2D(P3,T,R,A)
p3 = np.reshape(p3,(1,2))
p4 = projection3D2D(P4,T,R,A)
p4 = np.reshape(p4,(1,2))


points2D = np.concatenate((p1,p2,p3),axis=0);    # (3*2)
print("p1\n",p1)
print("pt2D\n",points2D)
print(np.shape(points2D))
# Application de solveP3P pour retrouver la position de la caméra
retval, rvec, tvecs =  cv2.solveP3P(points3D,points2D,A,None, flags = cv2.SOLVEPNP_P3P)

rvec = np.array(rvec)
tvecs = np.array(tvecs)


(1, 3)
P1
 [[-20.  20.   1.]]
pt3D
 [[-20.  20.   1.]
 [ 20. -20.   1.]
 [-20. -20.   1.]]
p1
 [[26.21935004 26.4859286 ]]
pt2D
 [[26.21935004 26.4859286 ]
 [19.70856451 12.74261826]
 [18.06715042 21.37146393]]
(3, 2)


In [186]:
# Calcul erreur estiamtion :
def distance(pt, pt_estimation):
    erreur = 0
    for i in range(len(pt)):
      erreur += (pt[i] - pt_estimation[i])**2
    return np.sqrt(erreur)

In [187]:


# Mesure de l'erreur d'estimation
nb_solutions = len(rvec)
print("nombre de solutions trouvées =", nb_solutions)

erreurs = []

for i in range (nb_solutions):
  print("\nSolution ",i, ":\n")
  # Récupération des matrices de rotation et translation

  rodriguez = rvec[i]     # (3*1)
  R_P3P = cv2.Rodrigues(rodriguez)[0]    # matrice de rotation : (3*3)
  print("R_P3P=",R_P3P,"\n")

  T_P3P = tvecs[i]    # matrice de translation : (1*3)
  T_P3P = np.reshape(T_P3P,(3,1))    # (3*1)
  print("T=",T_P3P,"\n")


  # Estimation des points
  p1_P3P = np.reshape(projection3D2D(P1,T_P3P,R_P3P,A),(1,2))

  p2_P3P = np.reshape(projection3D2D(P2,T_P3P,R_P3P,A),(1,2))
  p3_P3P = np.reshape(projection3D2D(P3,T_P3P,R_P3P,A),(1,2))
  p4_P3P = np.reshape(projection3D2D(P4,T_P3P,R_P3P,A),(1,2))

  # Création d'un tableau des points estimés
  pt_2D_P3P = np.concatenate((p1_P3P,p2_P3P,p3_P3P,p4_P3P),axis=0)    # (4,2)
  print("pt_2D_P3P = ",pt_2D_P3P,"\n")

  # Création d'un tableau des points originaux

  pt_2D = np.concatenate((p1,p2,p3,p4),axis=0)    # (4,2)
  print("pt_2D = ",pt_2D)

  # Calcul de l'erreur d'estimation
  erreurs.append([0])
  for j in range(len(pt_2D)):
    erreur_pt = distance(pt_2D[j],pt_2D_P3P[j])
    print("erreur = ",erreur_pt)
    erreurs[i]+=erreur_pt

# Récupérer la meilleure solution
indice_max = 0
max = erreurs[0]
for i in range(1,len(erreurs)) :
  if erreurs[i]>max :
    max = erreurs[i]
    indice_max = i

rodriguez = rvec[indice_max]
R_estime = cv2.Rodrigues(rodriguez)[0]
T_estime = tvecs[indice_max]    # matrice de translation : (1*3)
T_estime = np.reshape(T_P3P,(3,1))

print("\n R estimé = ", R_estime,"\n")
print("T estimé = ", T_estime, "\n")


nombre de solutions trouvées = 2

Solution  0 :

R_P3P= [[ 0.15487196  0.20968726  0.96542526]
 [-0.98457873 -0.04771731  0.16830857]
 [ 0.08135966 -0.97660346  0.19906354]] 

T= [[ 62.7467562 ]
 [ 79.29557946]
 [169.26704983]] 

pt_2D_P3P =  [[26.2193501  26.48592855]
 [19.70856457 12.7426182 ]
 [18.06715032 21.37146402]
 [28.10878308 15.52322168]] 

pt_2D =  [[26.21935004 26.4859286 ]
 [19.70856451 12.74261826]
 [18.06715042 21.37146393]
 [28.10878288 15.52322197]]
erreur =  7.373636208209806e-08
erreur =  8.284789123890252e-08
erreur =  1.3164376684733206e-07
erreur =  3.442116761869532e-07

Solution  1 :

R_P3P= [[ 0.33960444  0.65929874 -0.67081592]
 [-0.61962227  0.69339154  0.36779941]
 [ 0.70762777  0.29074618  0.64399503]] 

T= [[ 67.83204786]
 [ 84.82638078]
 [176.01615717]] 

pt_2D_P3P =  [[26.21935005 26.48592865]
 [19.70856446 12.74261832]
 [18.06715048 21.37146378]
 [26.59014756 17.6312078 ]] 

pt_2D =  [[26.21935004 26.4859286 ]
 [19.70856451 12.74261826]
 [18.06715042 2